# COVID-19 Analysis

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import datetime

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)


In [2]:

bingurl = 'https://raw.githubusercontent.com/microsoft/Bing-COVID-19-Data/master/data/Bing-COVID19-Data.csv'
bing = pd.read_csv(bingurl,index_col=0,parse_dates=[1])
df=bing
# df=bing[bing.Updated >= '3/01/2020']
# subset to only US and WW Region level without sub regions
df=df[df['Country_Region'].isin(['United States','Worldwide'])][df.AdminRegion1.isna()]
# create rolling 7 day average
df['ConfirmedChangeRolling'] = df.groupby('Country_Region')['ConfirmedChange'].rolling(7).mean().reset_index(0,drop=True)
df['DeathsChangeRolling'] = df.groupby('Country_Region')['DeathsChange'].rolling(7).mean().reset_index(0,drop=True)
df['RecoveredChangeRolling'] = df.groupby('Country_Region')['RecoveredChange'].rolling(7).mean().reset_index(0,drop=True)

df =df[df.Updated >= '3/01/2020']

In [3]:

USdf = df[df['Country_Region'].isin(['United States'])].copy()
USdf.index = USdf.Updated
WWdf = df[df['Country_Region'].isin(['Worldwide'])].copy()
WWdf.index = WWdf.Updated

In [4]:
fig = px.bar(USdf, x='Updated', y='ConfirmedChange')

fig.add_trace(go.Scatter(x=USdf.Updated, y=USdf.ConfirmedChangeRolling,
                         
                    mode='lines',
                    name='7 Day Rolling Average'))


fig.update_layout(
    height=800,
    xaxis_title="Time",
    yaxis_title="Daily Change in Confirmed Cases",
    title_text='USA Change in Confirmed COVID-19 Cases'
)
# fig.show()
iplot(fig)

In [5]:
fig = px.bar(WWdf, x='Updated', y='ConfirmedChange')

fig.add_trace(go.Scatter(x=WWdf.Updated, y=WWdf.ConfirmedChangeRolling,
                    mode='lines',
                    name='7 Day Rolling Average'))

fig.update_layout(
    height=800,
    xaxis_title="Time",
    yaxis_title="Daily Change in Confirmed Cases",
    title_text='Worldwide Change in Confirmed COVID-19 Cases'
)

# fig.show()
iplot(fig)

In [6]:
# DeathsChange data has some odd dips on certain Sundays, likely to delays in getting new numbers 
# or some other data management error. Using the rolling 7 day average makes for a cleaner trend
fig = px.line(df[df.AdminRegion1.isna()], x="Updated", y="DeathsChangeRolling", color='Country_Region')

fig.update_layout(
    height=600,
    xaxis_title="Time",
    yaxis_title="Daily Change in Deaths",
    title_text='Change in COVID-19 Deaths (USA vs World)'
)

# fig.show()
iplot(fig)

# Analysis

When comparing between the United States and Worldwide COVID-19 trends, we can see that worldwide confirmed cases increased more rapidly in other countries during March than in the United State. However, over the course of April and May, the United States had much higher rates of new daily confirmed cases and new daily death rate.  Interestingly, while deaths have trended downward worldwide, new cases have been trending upward. This trend is likely the result of increased testing capacity worldwide. Policy and social behavioral changes due to COVID-19 have affected regions in the US and other countries worldwide to aid in decreasing trends in deaths and US cases. As they come out of lockdown, concern is warranted that increased mobility by the populace may lead to additional outbreaks both in the US and elsewhere.